# Apprentissage sur les images

Entraine le modèle sur des images et pas sur les vidéos.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tensorflow import keras
from tensorflow.keras.preprocessing import image_dataset_from_directory

%matplotlib inline
plt.rcParams['figure.figsize'] = 12, 8
%config InlineBackend.figure_format = 'retina'

In [2]:
train_dataset = image_dataset_from_directory(
    directory='DB/train/',
    labels='inferred',
    label_mode='categorical',
    batch_size=32,
    image_size=(224, 224)
)

test_dataset = image_dataset_from_directory(
    directory="DB/test/",
    labels="inferred",
    label_mode="categorical",
    batch_size=32,
    image_size=(224, 224)
)

Found 4736 files belonging to 5 classes.
Found 3568 files belonging to 5 classes.


## Modèle de classification
Les images sont
- peu nombreuses
- parfois floues
- une main recouvre en partie les objets
- d'autres objets sont visibles en arrière plan

In [3]:
from keras.applications import VGG16
from keras.models import Sequential
from keras.layers import Input, Flatten, Dense, Dropout

vgg = VGG16(include_top=False, weights="imagenet")
vgg.trainable = False

modele = Sequential([
    Input((224, 224, 3)),
    vgg,
    Flatten(),
    Dense(1024, activation="elu"),
    Dropout(0.1),
    Dense(1024),
    Dense(5, activation="softmax")
])

modele.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Functional)           (None, None, None, 512)   14714688  
_________________________________________________________________
flatten (Flatten)            (None, 25088)             0         
_________________________________________________________________
dense (Dense)                (None, 1024)              25691136  
_________________________________________________________________
dropout (Dropout)            (None, 1024)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 1024)              1049600   
_________________________________________________________________
dense_2 (Dense)              (None, 5)                 5125      
Total params: 41,460,549
Trainable params: 26,745,861
Non-trainable params: 14,714,688
___________________________________

In [ ]:
from keras.optimizers import Adam
from keras.losses import categorical_crossentropy

optimiser = Adam()
modele.compile(optimizer=optimiser, loss=categorical_crossentropy, metrics=["accuracy"])
historique = modele.fit(train_dataset)
modele.save("modele-VGG16")